In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [9]:
stat_canada_data = pd.read_csv('../data-raw/linked_corp_data.csv')



In [10]:
stat_canada_data.shape

(67675, 16)

In [11]:
stat_canada_data1 = pd.read_csv('../data-raw/ICOCD_ICOE.csv')

In [14]:
stat_canada_data1.shape[0] - stat_canada_data.shape[0]

5898

In [2]:
cov_data = pd.read_csv("../data/raw/license_data_old.csv", sep=",", low_memory=False, dtype={20:'str'} )

In [3]:
cov_data = cov_data.assign(BusinessName_join = cov_data.BusinessName.str.lower().str.strip())
cov_data = cov_data.assign(BusinessTradeName_join = cov_data.BusinessTradeName.str.lower().str.strip())
cov_data = cov_data.assign(BusinessName_join = cov_data.BusinessName_join.str.replace(r"[^\w]", "", regex=True))
cov_data = cov_data.assign(BusinessTradeName_join = cov_data.BusinessTradeName_join.str.replace(r"[^\w]", "", regex=True))

In [4]:
stat_canada_data = pd.read_csv('../data/raw/linked_corp_data.csv')
stat_canada_data.columns = stat_canada_data.columns.str.strip()
stat_canada_data['IC_ID'] = 'C' + stat_canada_data.CCID.astype(str) + stat_canada_data.PID.astype(str)
stat_canada_data['NAME_JOIN'] = stat_canada_data['NAME'].str.lower().str.strip()
stat_canada_data['NAME_JOIN'] = stat_canada_data['NAME_JOIN'].str.replace(r'[^\w]', '', regex=True)

In [5]:
# create rankings
stat_canada_data_ranked = stat_canada_data
stat_canada_data_ranked = stat_canada_data_ranked.assign(
    ranking=stat_canada_data_ranked.groupby("NAME_JOIN")["COP"]
    .rank(ascending=False)
    .astype("int")
)
stat_canada_data_ranked = stat_canada_data_ranked.query("ranking ==1")
stat_canada_data_ranked = stat_canada_data_ranked.assign(
    ranking=stat_canada_data_ranked.groupby("NAME_JOIN")["LEVEL"]
    .rank(ascending=False)
    .astype("int")
)
stat_canada_data_ranked = stat_canada_data_ranked.query("ranking ==1")
stat_canada_data_ranked = stat_canada_data_ranked.assign(
    ranking=stat_canada_data_ranked.groupby("NAME_JOIN")["RECID"]
    .rank(ascending=False)
    .astype("int")
)
stat_canada_data_ranked = stat_canada_data_ranked.query("ranking ==1")

In [6]:
cov_data.shape

(1528385, 27)

In [7]:
# merge both dataframes
merged_df = cov_data.merge(stat_canada_data_ranked, how='left', left_on='BusinessName_join', right_on='NAME_JOIN')
merged_BN_df = merged_df[merged_df.NAME_JOIN.isnull()==False]
merged_BN_df.shape

(37824, 46)

In [8]:
assert merged_df.shape[0] == cov_data.shape[0], "Check Data inflation"

In [9]:
# try remaining unmatched on second name column
cov_data_remainder = cov_data[merged_df.NAME_JOIN.isnull()]

In [10]:
cov_data_remainder

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,Country,PostalCode,LocalArea,NumberofEmployees,FeePaid,ExtractDate,Geom,id,BusinessName_join,BusinessTradeName_join
0,11.0,1534378,11-108365,0.0,(Katherine Plett),NaN,Issued,2011-01-06,2011-12-31,One-Family Dwelling,...,CA,NaN,Kensington-Cedar Cottage,000,60.0,2019-07-21T06:48:58-07:00,NaN,B1,katherineplett,NaN
1,11.0,1534384,11-108371,0.0,(Maria Lowe),NaN,Issued,2011-01-17,2011-12-31,One-Family Dwelling,...,CA,NaN,Kitsilano,000,100.0,2019-07-21T06:48:58-07:00,NaN,B2,marialowe,NaN
2,11.0,1534389,11-108376,0.0,Bryant Tsz-Yeung Wong (Bryant Wong),NaN,Issued,2011-01-04,2011-12-31,One-Family Dwelling,...,CA,NaN,West Point Grey,000,60.0,2019-07-21T06:48:58-07:00,NaN,B3,bryanttszyeungwongbryantwong,NaN
3,11.0,1534395,11-108382,0.0,0796743 BC Ltd,NaN,Gone Out of Business,NaN,NaN,Multiple Dwelling,...,CA,NaN,Mount Pleasant,000,NaN,2019-07-21T06:48:58-07:00,NaN,B4,0796743bcltd,NaN
4,11.0,1534432,11-108419,0.0,Anka Investments Ltd,NaN,Issued,2010-12-22,2011-12-31,Apartment House,...,CA,V6J 3M8,Kitsilano,000,620.0,2019-07-21T06:48:58-07:00,"{""type"": ""Point"", ""coordinates"": [-123.1478691...",B5,ankainvestmentsltd,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1528380,21.0,3801719,21-205053,0.0,Deadly Couture Inc,NaN,Pending,NaN,NaN,Retail Dealer,...,CA,V5L 2G8,Grandview-Woodland,7,202.0,2021-03-13T00:08:41-08:00,"{""type"": ""Point"", ""coordinates"": [-123.0729908...",B569471,deadlycoutureinc,NaN
1528381,21.0,3801752,21-122322,1.0,Gavin Yu Houng Siu & Eva Paulina Siu,Empauer,Pending,NaN,NaN,Homecraft,...,CA,NaN,Hastings-Sunrise,2,NaN,2021-03-13T00:08:41-08:00,NaN,B569472,gavinyuhoungsiuevapaulinasiu,empauer
1528382,21.0,3801849,21-133522,1.0,Lana Porter Group Inc,NaN,Pending,NaN,NaN,Office,...,CA,V6B 2Y5,Downtown,1,NaN,2021-03-13T00:08:41-08:00,"{""type"": ""Point"", ""coordinates"": [-123.1235407...",B569473,lanaportergroupinc,NaN
1528383,21.0,3801864,21-137949,1.0,Coric Law Corporation,Coric Law,Pending,NaN,NaN,Office,...,CA,V6E 2Y3,West End,3,NaN,2021-03-13T00:08:41-08:00,"{""type"": ""Point"", ""coordinates"": [-123.1220088...",B569474,coriclawcorporation,coriclaw


In [11]:
merged_BTN_df = cov_data_remainder.merge(stat_canada_data_ranked, how='inner', left_on='BusinessTradeName_join', right_on='NAME_JOIN')

In [14]:
merged_df = pd.concat([merged_BN_df,merged_BTN_df], axis=0)
merged_df.columns

Index(['FOLDERYEAR', 'LicenceRSN', 'LicenceNumber', 'LicenceRevisionNumber',
       'BusinessName', 'BusinessTradeName', 'Status', 'IssuedDate',
       'ExpiredDate', 'BusinessType', 'BusinessSubType', 'Unit', 'UnitType',
       'House', 'Street', 'City', 'Province', 'Country', 'PostalCode',
       'LocalArea', 'NumberofEmployees', 'FeePaid', 'ExtractDate', 'Geom',
       'id', 'BusinessName_join', 'BusinessTradeName_join', 'CCID', 'LINKAGE',
       'PID', 'NAICS', 'CCTL', 'CRES', 'PRES', 'TYPE', 'NAME', 'COP',
       'RECTYPE', 'COMMENT', 'LEVEL', 'RQSTID', 'RECID', 'BN9', 'IC_ID',
       'NAME_JOIN', 'ranking'],
      dtype='object')

In [15]:
merged_df

,FOLDERYEAR,LicenceRSN,LicenceNumber,LicenceRevisionNumber,BusinessName,BusinessTradeName,Status,IssuedDate,ExpiredDate,BusinessType,...,COP,RECTYPE,COMMENT,LEVEL,RQSTID,RECID,BN9,IC_ID,NAME_JOIN,ranking
246,11.0,1535573,11-109560,0.0,Marumi Canada Lumber Ltd,NaN,Issued,2010-12-20,2011-12-31,One-Family Dwelling,...,100.0,C,FOE ...,2.0,4710.0,58786.0,0.0,C2092022910000795,marumicanadalumberltd,1.0
306,11.0,1535764,11-109745,0.0,Listel Canada Ltd,O'Doul's Outside Catering,Issued,2011-01-10,2011-12-31,Caterer,...,100.0,C,FOE ...,2.0,3477.0,43784.0,0.0,C8552081587132494,listelcanadaltd,1.0
702,9.0,1314376,09-120474,0.0,Sun Life Assurance Company Of Canada,NaN,Issued,2009-01-07,2009-12-31,Financial Services,...,100.0,C,...,2.0,1490.0,18045.0,0.0,C8500329189330492,sunlifeassurancecompanyofcanada,1.0
717,9.0,1314416,09-120514,0.0,HSBC Global Asset Management (Canada) Limited,NaN,Issued,2009-02-27,2009-12-31,Financial Services,...,100.0,C,FOE ...,4.0,324.0,3833.0,0.0,C5580900887158523,hsbcglobalassetmanagementcanadalimited,1.0
769,9.0,1314574,09-120672,0.0,Jarislowsky Fraser Limited,NaN,Issued,2009-01-02,2009-12-31,Financial Services,...,100.0,C,...,2.0,3205.0,40106.0,0.0,C8922003280131709,jarislowskyfraserlimited,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3108,14.0,2147403,14-134993,0.0,Western Coal Corp & Walter Canadian Coal ULC,Walter Canadian Coal Partnership,Inactive,2013-11-12T15:45:26-08:00,2014-12-31,Office,...,50.0,C,R ...,6.0,1837.0,21779.0,0.0,C9000058990194051,waltercanadiancoalpartnership,1.0
3109,20.0,3549264,20-105098,1.0,BentallGreenOak (Canada) GP Ltd,BentallGreenOak (Canada) Limited Partnership,Issued,2020-01-20T17:05:07-08:00,2020-12-31,Office,...,99.0,C,...,4.0,1490.0,18108.0,0.0,C8962804489330492,bentallgreenoakcanadalimitedpartnership,1.0
3110,21.0,3717199,21-134683,0.0,BentallGreenOak (Canada) GP Ltd,BentallGreenOak (Canada) Limited Partnership,Issued,2020-12-09T18:01:04-08:00,2021-12-31,Office,...,99.0,C,...,4.0,1490.0,18108.0,0.0,C8962804489330492,bentallgreenoakcanadalimitedpartnership,1.0
3111,20.0,3549256,20-184002,0.0,BentallGreenOak (Canada) GP Ltd,BentallGreenOak (Canada) Limited Partnership,Issued,2020-01-20T17:05:07-08:00,2020-12-31,Office,...,99.0,C,...,4.0,1490.0,18108.0,0.0,C8962804489330492,bentallgreenoakcanadalimitedpartnership,1.0
